In [43]:
import numpy as np

%reload_ext autoreload
%autoreload 2

from MTPHandler.core.plate import Plate
from MTPHandler.core.reactant import Reactant
from MTPHandler.core.protein import Protein
from MTPHandler.examples import parse_magellan # custom parser for SpectraMax plate reader

In [44]:
# Path to spectrometer output file
path = "Laccase_kinetic_woTween_Rawdata.xlsx"

# Load plate
plate = Plate.from_reader(
    reader=parse_magellan,
    path=path,
    ph=3,
    wavelength=420,
)

In [45]:
plate.visualize()

### Define reactants

In [46]:
buffer = plate.add_reactant(id="s0", name="Buffer", constant=True)

abts = plate.add_reactant(id="s1", name="ABTS", constant=False)

abts_radical = plate.add_reactant(id="s2", name="ABTS radical", constant=False)

slac = plate.add_protein(
    id="p0", name="Laccase", constant=True, sequence="MSSKSKPKDVKV"
)

### Assign initial conditions

In [ ]:
# Assign buffer concentration
plate.assign_species(to="all", species=buffer,
                     init_conc=100, conc_unit="mmol / l")

# Assign substrate concentrations
plate.assign_species(
    to="rows",
    species=abts,
    ids=["A", "B", "C", "D"],
    init_conc=[0, 0.3, 0.6, 0.9, 5, 30, 50, 70],
    conc_unit="mmol / l",
)

plate.assign_species(
    to="rows",
    species=abts,
    ids=["E", "F", "G", "H"],
    init_conc=[0.15, 0.45, 0.75, 1, 10, 40, 60, 80],
    conc_unit="mmol / l",
)

plate.assign_species(to="all", species=abts_radical,
                     init_conc=0, conc_unit="mmol / l")


# Assign enzyme concentrations
plate.assign_species(to="all", species=slac,
                     init_conc=0.006, conc_unit="mmol / l")

Assigned Buffer to all wells.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row A.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row B.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row C.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row D.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row E.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row F.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row G.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row H.
Assigned ABTS radical to all wells.
Assigned Laccase to all wells.


### Create EnzymeML document

In [ ]:
enzymeml = plate.to_enzymeml(
    name="laccase kinetic assay",
    detected_reactant=abts_radical,
    wavelength=420,
    path="lac_kinetics.json",
    ignore_blank_status=True,
)

Found 64 catalyzed wells


In [ ]:
def calculate_concentration(absorptions, ext_coeff: float, pathlength: float):
    return [absorption / (ext_coeff * pathlength) for absorption in absorptions]


def apply_extinction_coef(enzymeml, ext_coeff: float, pathlength: float):
    for measurement in enzymeml.measurements:
        for species in measurement.species:
            if not species.replicates:
                continue
            for replicate in species.replicates:
                replicate.data = calculate_concentration(
                    replicate.data, ext_coeff, pathlength
                )

In [ ]:
apply_extinction_coef(enzymeml, 36, 0.4)

In [ ]:
print(enzymeml)

EnzymeMLDocument
├── id = enzymemldocument1
├── name = laccase kinetic assay
├── created = 2023-10-30 21:08:31.715058
├── vessels
│   └── 0
│       └── Vessel
│           ├── id = plate0
│           ├── name = MTP 96 well
│           ├── volume = 200.0
│           ├── unit = ul
│           └── constant = True
├── proteins
│   └── 0
│       └── Protein
│           ├── id = p0
│           ├── name = Laccase
│           ├── vessel_id = plate0
│           ├── constant = True
│           ├── sequence = MSSKSKPKDVKV
│           └── ontology = SBO:0000013
├── reactants
│   ├── 0
│   │   └── Reactant
│   │       ├── id = s0
│   │       ├── name = Buffer
│   │       ├── vessel_id = plate0
│   │       ├── constant = True
│   │       └── ontology = SBO:0000247
│   ├── 1
│   │   └── Reactant
│   │       ├── id = s1
│   │       ├── name = ABTS
│   │       ├── vessel_id = plate0
│   │       ├── constant = False
│   │       └── ontology = SBO:0000247
│   └── 2
│       └── Reactant
│           ├── id 

In [ ]:
%reload_ext autoreload
%autoreload 2

from EnzymePynetics.core.estimator import Estimator


In [ ]:
estimator, _ = Estimator.from_enzymeml(enzymeml, abts_radical)

In [ ]:
oxidation = estimator.add_reaction(
    id="r1", name="Oxidation", educt=abts, product=abts_radical, enzyme=slac
)

In [ ]:
# Substrate rate laws
michaelis = estimator.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

competitive_product = estimator.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)",
)

substrate_inhibition = estimator.add_model(
    id="model4",
    name="substrate inhibition",
    equation="substrate = -k_cat * enzyme * substrate / (K_m + ((1+(substrate/K_iu))*substrate))",
)

# Enzyme rate laws
enzyme_inactivation = estimator.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="enzyme = -k_ie * enzyme ",
)

In [ ]:
# estimator.fit_models()

/Users/max/Documents/GitHub/software-driven-rdm/sdRDM/base/datamodel.py:326: UserWarning:

No 'URL' and 'Commit' specified. This model might not be re-usable.



Fitting michaelis-menten
Fitting michaelis-menten with enzyme inactivation
Fitting competitive product inhibition
Fitting competitive product inhibition with enzyme inactivation
Fitting substrate inhibition
Fitting substrate inhibition with enzyme inactivation


KeyboardInterrupt: 

In [ ]:
estimator.visualize()